# Dieselgate Routing

From the year 2019 on, Berlin will impose the Diesel ban. The following streets will be affected: Leipziger Straße, Reinhardstraße, Friedrichstraße, Brückenstraße, Kapweg, Alt-Moabit, Stromstraße und Leonorenstraße. A recent [Spiegel](https://www.spiegel.de) comic illustrated the issue:

![title](http://cdn3.spiegel.de/images/image-1349732-galleryV9-ikww-1349732.jpg)
© [SPIEGEL](https://www.spiegel.de)

This notebook explores the validity of the comic's claim. As a showcase, we'll have a look how the frequent visits of Angela Merkel to the German Currywurst Museum (solely inferred from superficial research) will change its route from 2019. You'll find remarkable similarities.

In [1]:
# Needed packages
from openrouteservice import client, geocode
import folium
from shapely.geometry import LineString, Polygon, mapping
from shapely.ops import cascaded_union
import time
import ipywidgets as widgets
from IPython.display import display

def style_function(color): # To style data
    return lambda feature: dict(color=color,
                                opacity=0.5,
                                weight=4,)

### Regular Route

So far: The shortest route for a car from A to B.

In [2]:
# Widget
input_start = widgets.Text(description = "Start:")
input_destination = widgets.Text(description = "Destination:")
display(input_start)
display(input_destination)

button = widgets.Button(description="Let's go!")
display(button)

Text(value='', description='Start:')

Text(value='', description='Destination:')

Button(description="Let's go!", style=ButtonStyle())

In [3]:
# Basic parameters
api_key = 'your_key' #https://openrouteservice.org/sign-up
clnt = client.Client(key=api_key)

map_berlin = folium.Map(tiles='https://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', 
                        attr='Map data (c) OpenStreetMap, Tiles (c) <a href="https://heigit.org">GIScience Heidelberg</a>', 
                        location=([52.516586, 13.381047]), 
                        zoom_start=13.5) # Create map

poi_start = geocode.pelias_search(client=clnt, text={input_start.value})
poi_dest = geocode.pelias_search(client=clnt, text={input_destination.value})


coords_start = []
for street in poi_start['features']:
    if street['properties']['county'] == 'Berlin ':
        coords_start.append(street['geometry']['coordinates'])
folium.Marker(list(reversed(coords_start[0])), popup='Start').add_to(map_berlin)
        
coords_dest = []
for street in poi_dest['features']:
    if street['properties']['county'] == 'Berlin ':
        coords_dest.append(street['geometry']['coordinates'])
folium.Marker(list(reversed(coords_dest[0])), popup='Destination').add_to(map_berlin)


# Regular Route
popup_route = "<h4>{0} route</h4><hr>" \
             "<strong>Duration: </strong>{1:.1f} mins<br>" \
             "<strong>Distance: </strong>{2:.3f} km" 

# Request route
direction_params = {'coordinates': [coords_start[0], coords_dest[0]],
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}

regular_route = clnt.directions(**direction_params) # Direction request

# Build popup
duration, distance = regular_route['features'][0]['properties']['summary'][0].values()
popup = folium.features.Popup(popup_route.format('Regular', 
                                                 duration/60, 
                                                 distance/1000))

gj= folium.GeoJson(regular_route,
                   name='Regular Route',
                   style_function=style_function('blue')).add_child(popup).add_to(map_berlin)

map_berlin

### Dieselgate Routing

Coming soon: The shortest route for a Diesel driver, which must avoid the blackish areas. Then, affected cars can't cross Friedrichstraße anymore. But see for yourself:

In [4]:
# Start and destination coordinates of affectes streets
avoid_streets = [{'name':'Friedrichstraße', 'coords': [[13.388633, 52.518095], [13.388547, 52.51863]]}, 
                 {'name': 'Leiziger Straße/Bundesrat', 'coords': [[13.384041, 52.510005], [13.381358, 52.509829]]}, 
                 {'name': 'Leiziger Straße/Friedrichstraße', 'coords': [[13.38963, 52.510338], [13.387538, 52.51022]]},
                 {'name': 'Leiziger Straße/Charlottenstraße', 'coords': [[13.39169, 52.510469], [13.390038, 52.510371]]},
                 {'name': 'Brückenstraße', 'coords': [[13.417912, 52.514876], [13.416538, 52.511135]]},
                 {'name': 'Kapweg', 'coords': [[13.326888, 52.562506], [13.330107, 52.562349]]},
                 {'name': 'Reinhardtstraße', 'coords': [[13.377249, 52.522188], [13.376616, 52.52207]]},
                 {'name': 'Reinhardtstraße', 'coords': [[13.377475, 52.52224], [13.379159, 52.522514]]},
                 {'name': 'Alt-Moabit', 'coords': [[13.32765, 52.524316], [13.329313, 52.523859]]}, 
                 {'name': 'Leonorenstraße', 'coords': [[13.345524, 52.436411], [13.344687, 52.437608]]},
                 {'name': 'Stromstraße', 'coords': [[13.343228, 52.526744], [13.343282, 52.528037]]}]

# Affected streets
buffer = []
for street in avoid_streets:
    avoid_params = {'coordinates': street['coords'],
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}
    avoid_request = clnt.directions(**avoid_params)
    coords = avoid_request['features'][0]['geometry']['coordinates']
    route_buffer = LineString(coords).buffer(0.0005) # Create geometry buffer
    folium.features.PolygonMarker([(y,x) for x,y in list(route_buffer.exterior.coords)], 
                                color=('#FF0000'), 
                                popup=street['name'],).add_to(map_berlin)
    simp_geom = route_buffer.simplify(0.005) # Simplify geometry for better handling
    buffer.append(simp_geom)
union_buffer = cascaded_union(buffer)
map_berlin

If they are serious about it, this will be the route while avoiding the banned areas:

In [5]:
# New routing with avoided streets
diesel_request = {'coordinates': [coords_start[0], coords_dest[0]], 
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': False,
                 'options': {'avoid_polygons': mapping(union_buffer)}} 
route_diesel = clnt.directions(**diesel_request)

# Build popup
duration, distance = route_diesel['features'][0]['properties']['summary'][0].values()
popup = folium.features.Popup(popup_route.format('Diesel Route',
                                                 duration/60,
                                                 distance/1000))

folium.GeoJson(route_diesel,
               style_function=style_function('red'), 
               name='Route after Jan 2019').add_child(popup).add_to(map_berlin)

map_berlin.add_child(folium.map.LayerControl())

Now, here it should be noted, that our dear Chancellor would have to drive a detour of more than **1.5 times** the current distance, imposing 50% more pollution on Berlin's residents, just to enjoy the history of the Currywurst. Click on the routes to see for yourself.

At least Friedrichstraße is safe soon!